### Losa 

분류

‘Toyota 서비스센터 이용 고객 데이터’ >  Elasticsearch의Machine Learning에 학습시켜 ‘EFA’ 옵션 사용을 예측

2. 환경 구성
  ⬝ 데이터
     ⬝ 학습 데이터 : 202208_rosa_training.xlsx (2020.08~2022.06)
     ⬝ 예측 대상 데이터 : 202208_rosa_training.xlsx (2022.07)
        ⬝ 분석용 데이터는 Feature를 포함하고 있어야 하기 때문에
           학습 데이터 중 일부를 분리해서 사용

3. 사전 작업
  ⬝ 데이터 정제
     ⬝ 202208_rosa_training의  ’EFA’ 값을 ML의 지도학습에 맞는 형태로 변경
        ⬝ None : 0
        ⬝ EFA : 1



인덱스를 만들고 머신러닝 잡을 만들 때 필드를 선택 -> 이 과정에서 인코딩을 해줌


In [4]:
import pandas as pd

df = pd.read_excel(r'C:\Users\lloydk-dahye\workspace\1_elastic_prj\3_machine_learning\202208_rosa_training.xlsx')
df

,CENTER,RO,CUST_SEQ,CUST_TYPE,CPS_3YR_PROPO,VISIT_3YR_PROPO,VIN,SALES_TYPE,BRAND,HEV,HP,TAKING_FIX,VEHICLE_AGE_PROPO,MILEAGE_PROPO,GR_BP,KPI_CATEGORY,12V,TIRE,EFA,Air Care
0,렉서스강남서비스센터,2020-08-01(1),1028718,기타 개인,2001670.0,6,JTHBK1EG3B2464744,일반 법인,L,0,277,5,8.9,108393,기능,PSP,NaN,NaN,EFA,NaN
1,렉서스강남서비스센터,2020-08-01(2),1745419,개인,0.0,3,JTJHJRDZ1L2129271,TFSKR 임직원,L,1,152,5,1.1,4728,기능,FMS,NaN,NaN,NaN,NaN
2,렉서스강남서비스센터,2020-08-01(3),47482,개인,1050390.0,4,JTHBJ46G072006122,일반 고객,L,0,277,5,14.3,165551,기능,GR,NaN,NaN,EFA,NaN
3,렉서스강남서비스센터,2020-08-01(4),1052522,법인,11762300.0,12,JTHDU1EF0C5014897,일반 고객,L,1,394,5,8.7,289222,기능,PM Others,NaN,NaN,EFA,NaN
4,렉서스강남서비스센터,2020-08-01(5),1678559,개인,0.0,4,JTHB21B12K2023719,재구매_본인,L,1,178,5,1.6,16269,기능,FMS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303123,렉서스해운대서비스센터,2022-07-29(681),1855409,개인,7040.0,4,JTHBA1C13M2004178,일반 고객,L,1,178,5,1.3,17914,기능,GR,NaN,NaN,NaN,NaN
303124,렉서스해운대서비스센터,2022-07-29(683),1333822,개인,2115410.0,10,JTJBGMCA2H2029450,일반 고객,L,1,262,5,4.9,86729,기능,PSP,NaN,NaN,EFA,NaN
303125,렉서스해운대서비스센터,2022-07-30(686),1455985,개인,1378080.0,9,JTJBJRBZ1H2052949,일반 고객,L,1,152,5,5.8,117471,기능,PSP,NaN,NaN,NaN,NaN
303126,렉서스해운대서비스센터,2022-07-30(687),1776893,개인,136950.0,4,JTHP6JBH4M2089067,일반 고객,L,1,146,5,1.5,29341,기능,FMS,NaN,NaN,NaN,NaN


차대번호
필드 제거
010101

In [8]:
from datetime import datetime

df['DATE'] = df['RO'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df['DATE'] = pd.to_datetime(df['DATE'])

In [10]:
특정값 = 10
df['EFA'] = (df['EFA'] == 'EFA').astype(int)

In [14]:
df['CHARACTERISTICS'] = df['VIN'].str[3:9]
df['MODEL_YEAR'] = df['VIN'].str[10]


In [15]:
df.columns

Index(['CENTER', 'RO', 'CUST_SEQ', 'CUST_TYPE', 'CPS_3YR_PROPO',
       'VISIT_3YR_PROPO', 'VIN', 'SALES_TYPE', 'BRAND', 'HEV', 'HP',
       'TAKING_FIX', 'VEHICLE_AGE_PROPO', 'MILEAGE_PROPO', 'GR_BP',
       'KPI_CATEGORY', '12V', 'TIRE', 'EFA', 'Air Care', 'DATE',
       'CHARACTERISTICS', 'MODEL_YEAR'],
      dtype='object')

In [18]:
new_df = df.drop(columns=['12V', 'TIRE', 'Air Care', 'RO'])

In [22]:
new_df.to_csv('rosa_pre_data.csv', encoding='utf-8', index=False)

In [25]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303128 entries, 0 to 303127
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   CENTER             303128 non-null  object        
 1   CUST_SEQ           303128 non-null  int64         
 2   CUST_TYPE          303128 non-null  object        
 3   CPS_3YR_PROPO      303128 non-null  float64       
 4   VISIT_3YR_PROPO    303128 non-null  int64         
 5   VIN                303128 non-null  object        
 6   SALES_TYPE         302980 non-null  object        
 7   BRAND              303128 non-null  object        
 8   HEV                303128 non-null  int64         
 9   HP                 303128 non-null  int64         
 10  TAKING_FIX         303128 non-null  int64         
 11  VEHICLE_AGE_PROPO  303128 non-null  float64       
 12  MILEAGE_PROPO      303128 non-null  int64         
 13  GR_BP              303128 non-null  object  

 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   CENTER    서비스         303128 non-null  object        
 1   CUST_SEQ     고객번호      303128 non-null  int64         
 2   CUST_TYPE    고객유형      303128 non-null  object        
 3   CPS_3YR_PROPO   3년유상금액   303128 non-null  float64       
 4   VISIT_3YR_PROPO  3년방문횟수  303128 non-null  int64         
 5   VIN            차대번호    303128 non-null  object        
 6   SALES_TYPE     판매유형    302980 non-null  object        
 7   BRAND          브랜드    303128 non-null  object        
 8   HEV            하이브리드여부    303128 non-null  int64         
 9   HP             엔진출력    303128 non-null  int64         
 10  TAKING_FIX      승차정원   303128 non-null  int64         
 11  VEHICLE_AGE_PROPO  차령(제안서) 303128 non-null  float64       
 12  MILEAGE_PROPO   주행거리(제안서)   303128 non-null  int64         
 13  GR_BP          수리작업구분    303128 non-null  object        
 14  KPI_CATEGORY     KPI구분  303128 non-null  object        
 15  EFA        예측컬럼 (0, 1)        303128 non-null  int32         
 16  DATE   날짜            303128 non-null  datetime64[ns]
 17  CHARACTERISTICS  차량 특성군  303128 non-null  object        
 18  MODEL_YEAR  차량연식(차대번호)       303128 non-null  object   

In [23]:
df = pd.read_csv('rosa_pre_data.csv')
df

,CENTER,CUST_SEQ,CUST_TYPE,CPS_3YR_PROPO,VISIT_3YR_PROPO,VIN,SALES_TYPE,BRAND,HEV,HP,TAKING_FIX,VEHICLE_AGE_PROPO,MILEAGE_PROPO,GR_BP,KPI_CATEGORY,EFA,DATE,CHARACTERISTICS,MODEL_YEAR
0,렉서스강남서비스센터,1028718,기타 개인,2001670.0,6,JTHBK1EG3B2464744,일반 법인,L,0,277,5,8.9,108393,기능,PSP,1,2020-08-01,BK1EG3,2
1,렉서스강남서비스센터,1745419,개인,0.0,3,JTJHJRDZ1L2129271,TFSKR 임직원,L,1,152,5,1.1,4728,기능,FMS,0,2020-08-01,HJRDZ1,2
2,렉서스강남서비스센터,47482,개인,1050390.0,4,JTHBJ46G072006122,일반 고객,L,0,277,5,14.3,165551,기능,GR,1,2020-08-01,BJ46G0,2
3,렉서스강남서비스센터,1052522,법인,11762300.0,12,JTHDU1EF0C5014897,일반 고객,L,1,394,5,8.7,289222,기능,PM Others,1,2020-08-01,DU1EF0,5
4,렉서스강남서비스센터,1678559,개인,0.0,4,JTHB21B12K2023719,재구매_본인,L,1,178,5,1.6,16269,기능,FMS,0,2020-08-01,B21B12,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303123,렉서스해운대서비스센터,1855409,개인,7040.0,4,JTHBA1C13M2004178,일반 고객,L,1,178,5,1.3,17914,기능,GR,0,2022-07-29,BA1C13,2
303124,렉서스해운대서비스센터,1333822,개인,2115410.0,10,JTJBGMCA2H2029450,일반 고객,L,1,262,5,4.9,86729,기능,PSP,1,2022-07-29,BGMCA2,2
303125,렉서스해운대서비스센터,1455985,개인,1378080.0,9,JTJBJRBZ1H2052949,일반 고객,L,1,152,5,5.8,117471,기능,PSP,0,2022-07-30,BJRBZ1,2
303126,렉서스해운대서비스센터,1776893,개인,136950.0,4,JTHP6JBH4M2089067,일반 고객,L,1,146,5,1.5,29341,기능,FMS,0,2022-07-30,P6JBH4,2
